In [1]:
def ma(lead, alpha, past_order, inv, on_the_way):
    return int(lead*alpha* np.mean(past_order)) - inv - on_the_way

In [11]:
def simulator_date(table, alpha, window, func, LEAD = 12, INIT_AMOUNT = 0, pr = True):
    import numpy as np
    import pandas as pd
    from datetime import datetime
    from datetime import timedelta
    
    LEAD_WEEK = timedelta(days = 7*LEAD)
    
    if INIT_AMOUNT == 0:
        INIT_AMOUNT = np.mean(table["OrderQty"]) * LEAD * 1.2  #buffer??
    # Duplicated table
    copy = table.copy()
    copy["Inventory"] = np.zeros(copy.shape[0])
    copy["Inventory"].iloc[window] = INIT_AMOUNT
    copy["ShippedQty"] = np.zeros(copy.shape[0])
    copy["ArrivalQty"] = np.zeros(copy.shape[0])
    copy["RefillQty"] = np.zeros(copy.shape[0])
    
    # Initialization
    refill_schedules = []
    on_the_way = 0
    inv = INIT_AMOUNT
    results = list()
    for day in copy.index[window:]:
        # 1. Arrival
        if refill_schedules != []:
            if refill_schedules[0][0]<= day:
                inv += refill_schedules[0][1]
                copy.loc[day,"ArrivalQty"] = refill_schedules[0][1]
                on_the_way -= refill_schedules[0][1]
                refill_schedules.pop(0)

        # 2. Order
        order= copy.loc[ day, "OrderQty"]

        # 3. Ship
        if inv < order:
            copy.loc[day,"ShippedQty"] = inv
            inv = 0
        else:
            copy.loc[ day, "ShippedQty"] = order
            inv -= order

        # 4. Refill
        past_order = copy["OrderQty"][day-timedelta(days = 7*window):day]
        refill = func(LEAD, alpha, past_order, inv, on_the_way)
        if refill > 0 :
            on_the_way += refill
            copy.loc[day,"RefillQty"] = refill
            refill_schedules.append((day+LEAD_WEEK,refill) )
        
        # 5. Inventory - audit
        copy.loc[ day,"Inventory"] = inv
        
    # Report :
    avg = np.mean(table["OrderQty"])
    std = np.std(table["OrderQty"])
    #fulfill = np.mean(copy["ShippedQty"][window:]/copy["OrderQty"][window:])
    rate = sum(copy["ShippedQty"][window:])/sum(copy["OrderQty"][window:])
    maxInv = max(copy["Inventory"][window+LEAD:])
    avgInv = np.mean(copy["Inventory"][window+LEAD:])
    
#     if pr:
#         print("Entering simulator with parameter alpha: %s, window: %d, function: %s "%( alpha, window, func.__name__))
#         print("Average Order Qty: %d, Standard deviation: %.2f"%(avg, std ))
#         print("Fulfillment rate: %.3f" %(rate))
#         #print("Average rate: %.3f" %(rate))
#         print("Maximum Inventory occupation: %d" %maxInv)
#         print("Average Inventory occupation: %.0f"% avgInv)
        
    return avg, std, rate, maxInv, avgInv#, copy["Inventory"]

In [31]:
#item = 10239

In [78]:
item_list = pd.read_csv('item_list.csv',header=None)

In [87]:
item_l = item_list.values.tolist()[0]  #item list 

In [92]:
a_list = np.arange(0.1, 10.1, 0.1)

import csv
import pandas as pd
import numpy as np
from datetime import datetime
def parser(x):
    return datetime.strptime(x,"%Y-%m-%d")

for item in item_l:
    
    data = pd.read_csv("itemByWeek/{}ByWeek.csv".format(item), parse_dates=[0], index_col=0, date_parser=parser)
    data["OrderQty"] = [float(x) for x in data["OrderQty"]]
    #alpha = 1.3
    #window = 15

    for alpha in a_list:
        for window in range(1,51):
            R95 = 0
            R98 = 0
            R100 = 0
            Order_QTY,STD,FR,Max_Inv,Avg_Inv = simulator_date(data, alpha, window, ma) 
            if FR >= 1 and R100 == 0:
                AVG100 = Avg_Inv
                MAX100 = Max_Inv
                R100 = FR
                if R95 == 0:
                    AVG95 = Avg_Inv
                    MAX95 = Max_Inv
                    R95 = FR

                if R98 == 0:
                    AVG98 = Avg_Inv
                    MAX98 = Max_Inv
                    R98 = FR


            elif FR >= 0.98 and FR < 1:
                if R98 == 0:
                    AVG98 = Avg_Inv
                    MAX98 = Max_Inv
                    R98 = FR
                    if R95 == 0:
                        AVG95 = Avg_Inv
                        MAX95 = Max_Inv
                        R95 = FR


            elif FR >= 0.95 and FR < 0.98:
                if R95 == 0:
                    AVG95 = Avg_Inv
                    MAX95 = Max_Inv
                    R95 = FR

            if R95 == 0:
                AVG95 = 'MA cannot achieve this fulfillment rate. Please try other models.'
                MAX95 = ''

            if R98 == 0:
                AVG98 = 'MA cannot achieve this fulfillment rate. Please try other models.'
                MAX98 = ''

            if R100 == 0:
                AVG100 = 'MA cannot achieve this fulfillment rate. Please try other models.'
                MAX100 = ''

            with open("MA_%.0f.csv" %item, 'a', newline='') as f:
                spamwriter = csv.writer(f)
                spamwriter.writerow(['Fulfillment Rate','Avg Inventory','Max Inventory','Alpha','Window'])
                spamwriter.writerow(['95%', AVG95,MAX95,alpha,window])
                spamwriter.writerow(['98%', AVG98,MAX98,alpha,window])
                spamwriter.writerow(['100%', AVG100,MAX100,alpha,window])

Entering simulator with parameter alpha: 0.0, window: 1, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.093
Maximum Inventory occupation: 0
Average Inventory occupation: 0
Entering simulator with parameter alpha: 0.0, window: 2, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.093
Maximum Inventory occupation: 0
Average Inventory occupation: 0
Entering simulator with parameter alpha: 0.0, window: 3, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.094
Maximum Inventory occupation: 0
Average Inventory occupation: 0
Entering simulator with parameter alpha: 0.0, window: 4, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.094
Maximum Inventory occupation: 0
Average Inventory occupation: 0
Entering simulator with parameter alpha: 0.0, window: 5, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.094
Maximum I

Entering simulator with parameter alpha: 0.0, window: 41, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.131
Maximum Inventory occupation: 1073
Average Inventory occupation: 10
Entering simulator with parameter alpha: 0.0, window: 42, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.132
Maximum Inventory occupation: 351
Average Inventory occupation: 3
Entering simulator with parameter alpha: 0.0, window: 43, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.134
Maximum Inventory occupation: 1391
Average Inventory occupation: 22
Entering simulator with parameter alpha: 0.0, window: 44, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.135
Maximum Inventory occupation: 2003
Average Inventory occupation: 28
Entering simulator with parameter alpha: 0.0, window: 45, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment ra

Entering simulator with parameter alpha: 0.1, window: 30, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.208
Maximum Inventory occupation: 1097
Average Inventory occupation: 14
Entering simulator with parameter alpha: 0.1, window: 31, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.210
Maximum Inventory occupation: 986
Average Inventory occupation: 13
Entering simulator with parameter alpha: 0.1, window: 32, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.212
Maximum Inventory occupation: 1156
Average Inventory occupation: 15
Entering simulator with parameter alpha: 0.1, window: 33, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.213
Maximum Inventory occupation: 767
Average Inventory occupation: 12
Entering simulator with parameter alpha: 0.1, window: 34, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment ra

Entering simulator with parameter alpha: 0.2, window: 19, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.289
Maximum Inventory occupation: 699
Average Inventory occupation: 20
Entering simulator with parameter alpha: 0.2, window: 20, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.291
Maximum Inventory occupation: 1073
Average Inventory occupation: 33
Entering simulator with parameter alpha: 0.2, window: 21, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.291
Maximum Inventory occupation: 721
Average Inventory occupation: 25
Entering simulator with parameter alpha: 0.2, window: 22, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.287
Maximum Inventory occupation: 987
Average Inventory occupation: 38
Entering simulator with parameter alpha: 0.2, window: 23, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rat

Entering simulator with parameter alpha: 0.3, window: 9, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.365
Maximum Inventory occupation: 3508
Average Inventory occupation: 151
Entering simulator with parameter alpha: 0.3, window: 10, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.367
Maximum Inventory occupation: 3018
Average Inventory occupation: 131
Entering simulator with parameter alpha: 0.3, window: 11, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.368
Maximum Inventory occupation: 2459
Average Inventory occupation: 126
Entering simulator with parameter alpha: 0.3, window: 12, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.368
Maximum Inventory occupation: 1191
Average Inventory occupation: 92
Entering simulator with parameter alpha: 0.3, window: 13, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillmen

...

Entering simulator with parameter alpha: 9.6, window: 19, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 169072
Average Inventory occupation: 131100
Entering simulator with parameter alpha: 9.6, window: 20, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 165539
Average Inventory occupation: 129481
Entering simulator with parameter alpha: 9.6, window: 21, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 164664
Average Inventory occupation: 128307
Entering simulator with parameter alpha: 9.6, window: 22, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 161364
Average Inventory occupation: 127273
Entering simulator with parameter alpha: 9.6, window: 23, function: ma 
Average Order Qty: 1157, Standard deviat

Entering simulator with parameter alpha: 9.7, window: 6, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.979
Maximum Inventory occupation: 204757
Average Inventory occupation: 153528
Entering simulator with parameter alpha: 9.7, window: 7, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.974
Maximum Inventory occupation: 200484
Average Inventory occupation: 151315
Entering simulator with parameter alpha: 9.7, window: 8, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.970
Maximum Inventory occupation: 205038
Average Inventory occupation: 151772
Entering simulator with parameter alpha: 9.7, window: 9, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 0.976
Maximum Inventory occupation: 206435
Average Inventory occupation: 151504
Entering simulator with parameter alpha: 9.7, window: 10, function: ma 
Average Order Qty: 1157, Standard deviation:

Entering simulator with parameter alpha: 9.7, window: 43, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 149127
Average Inventory occupation: 124189
Entering simulator with parameter alpha: 9.7, window: 44, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 147316
Average Inventory occupation: 123756
Entering simulator with parameter alpha: 9.7, window: 45, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 147195
Average Inventory occupation: 123750
Entering simulator with parameter alpha: 9.7, window: 46, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 146352
Average Inventory occupation: 123647
Entering simulator with parameter alpha: 9.7, window: 47, function: ma 
Average Order Qty: 1157, Standard deviat

Entering simulator with parameter alpha: 9.8, window: 31, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 153176
Average Inventory occupation: 127803
Entering simulator with parameter alpha: 9.8, window: 32, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 154933
Average Inventory occupation: 127909
Entering simulator with parameter alpha: 9.8, window: 33, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 154960
Average Inventory occupation: 127982
Entering simulator with parameter alpha: 9.8, window: 34, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 155041
Average Inventory occupation: 127775
Entering simulator with parameter alpha: 9.8, window: 35, function: ma 
Average Order Qty: 1157, Standard deviat

Average Inventory occupation: 137962
Entering simulator with parameter alpha: 9.9, window: 19, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 174784
Average Inventory occupation: 135950
Entering simulator with parameter alpha: 9.9, window: 20, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 171169
Average Inventory occupation: 134253
Entering simulator with parameter alpha: 9.9, window: 21, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 170242
Average Inventory occupation: 133033
Entering simulator with parameter alpha: 9.9, window: 22, function: ma 
Average Order Qty: 1157, Standard deviation: 432.36
Fulfillment rate: 1.000
Maximum Inventory occupation: 166861
Average Inventory occupation: 131964
Entering simulator with parameter alpha: 9.9, window: 23, function: ma 
Ave